## Speck32 Encryption Algorithm

In [1]:
import random
import numpy as np

In [2]:
def speck32_encrypt(plaintext, key):
    R = 22
    w = 16
    K = [key & 0xFFFF, (key >> 16) & 0xFFFF]
    x = plaintext & 0xFFFF
    y = (plaintext >> 16) & 0xFFFF
    round_constants = [(i + 1) & 0xFFFF for i in range(R)]
    for i in range(R):
        x = (x + K[0]) & 0xFFFF
        x = ((x << 8) | (x >> 8)) & 0xFFFF
        x = (x ^ y) & 0xFFFF
        y = ((y << 3) | (y >> (w - 3))) & 0xFFFF
        y = (y ^ x) & 0xFFFF
        K[0] = (K[0] + round_constants[i]) & 0xFFFF
    return (y << 16) | x

## Generating Keystream

In [3]:
key = random.getrandbits(32)
with open('speck32_keystream.bin', 'wb') as f:
    for _ in range(10000): 
        keystream_block = speck32_encrypt(random.getrandbits(32), key)
        f.write(keystream_block.to_bytes(4, 'little'))

## Encrypting the Message

In [4]:
plaintext = "My name is Khushi"
plaintext_bytes = bytearray(plaintext, 'utf-8')

In [5]:
with open('speck32_keystream.bin', 'rb') as f:
    keystream = f.read(len(plaintext_bytes))

In [6]:
encrypted_bytes = bytearray(a ^ b for a, b in zip(plaintext_bytes, keystream))

In [7]:
with open('encrypted_text.bin', 'wb') as f:
    f.write(encrypted_bytes)

In [8]:
print("Original plaintext:", plaintext)

Original plaintext: My name is Khushi


In [10]:
print("Encrypted bytes:", encrypted_bytes)

Encrypted bytes: bytearray(b'\t\rA\xc2x\xe6\xe3\x86\xc4\x13s\xc3\xff\xc7\x05\xf8\x03')


## Decryption Process

In [11]:
with open('speck32_keystream.bin', 'rb') as f:
    keystream = f.read(len(plaintext_bytes))

In [12]:
with open('encrypted_text.bin', 'rb') as f:
    encrypted_bytes = f.read()

In [13]:
decrypted_bytes = bytearray(a ^ b for a, b in zip(encrypted_bytes, keystream))

In [14]:
decrypted_text = decrypted_bytes.decode('utf-8')

In [15]:
print("Decrypted text:", decrypted_text)

Decrypted text: My name is Khushi
